In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point
from mpl_toolkits.basemap import Basemap
import glob
import rasterio
import os
import shutil
import pyimpute
from pathlib import Path

In [ ]:
import fiona

# Vamos gerar os pontos de presença

In [ ]:
def check_if_feature_contain_property(feat, key, value) -> bool:
    prop = dict(feat.properties.items())
    if prop[key] == value:
        return True
    else:
        return False

def check_if_feature_property_is_in(feat, key, values: list)->bool:
    prop = dict(feat.properties.items())
    if prop[key] in values:
        return True
    else:
        return False

Params:
- STATES = ["RS","SC","PR","SP","MG","ES","RJ"]

In [ ]:
STATES = ["RS","SC","PR","SP","MG","ES","RJ"]
#STATES = ["RS","SC","PR","SP","MG","ES"]

In [ ]:
# Source: IBGE:  bcim_2016_21_11_2018
import json

INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']]
INFOS_UFS.to_file("assets/UFS_JSON", driver = "GeoJSON")
with open("assets/UFS_JSON") as geofile:
    geojson_file = json.load(geofile)

    geojson_file: dict
        - type: str
        - crs: dict
        - features: list
          - feat_obj: dict
            - type: str
            - properties: dict
              - sigla: str
            - geometry: dict
              - type: str
              - coordinates: list[int, int]

## Filtering states
Queremos atualizar a lista features removendo aqueles em que a sigla não está na lista STATES

In [ ]:
brazil_states_filtered = [feat for feat in geojson_file['features'] if feat['properties']['sigla'] in STATES]
updated_brazil_states_filtered = {'type':geojson_file['type'],'features':brazil_states_filtered}

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Source: IBGE:  bcim_2016_21_11_2018
import json

INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']]
INFOS_UFS.to_file("assets/UFS_JSON", driver = "GeoJSON")

def generate_random_points(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

with open("assets/UFS_JSON") as geofile:
    geojson_file = json.load(geofile)
INFOS_UFS = INFOS_UFS[INFOS_UFS['sigla'].isin(STATES)].reset_index()
br_union_geo = INFOS_UFS['geometry'].unary_union

gdf_poly = gpd.GeoDataFrame(index=["myPoly"], geometry=[br_union_geo], crs = {'init': 'epsg:4326'},)

x, y = generate_random_points(br_union_geo, 6_000)

df = pd.DataFrame()
df['points'] = list(zip(x,y))
df['points'] = df['points'].apply(Point)
gdf_points = gpd.GeoDataFrame(df, geometry='points')
Sjoin = gpd.tools.sjoin(gdf_points, gdf_poly, predicate="within", how='left')

# Keep points in "myPoly"
absence = gdf_points[Sjoin.index_right=='myPoly']


In [ ]:
from urllib import request

url = 'https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/brazil-states.geojson'
with request.urlopen(url) as f:
    brazil_states = json.load(f)
    
brazil_states_filtered = [feat for feat in brazil_states['features'] if feat['properties']['sigla'] in STATES]
updated_brazil_states_filtered = {'type':brazil_states['type'],'features':brazil_states_filtered}

In [ ]:
#geojson_file_filtered = [feat for feat in geojson_file['features'] if feat['properties']['sigla'] in STATES]
#updated_geojson_file_filtered = {'type':geojson_file['type'],'features':geojson_file_filtered}

In [ ]:
'''
fig = px.choropleth(
        INFOS_UFS,
        locations="sigla",
        scope='south america',
        center={"lat": -16.95, "lon": -47.78},
        geojson=updated_geojson_file_filtered,
        featureidkey='properties.sigla', # add
        color_continuous_scale="Redor",
        hover_data={"sigla": True}
)
fig.update_geos(fitbounds="geojson", visible=False,framewidth=2, showframe=True)
fig.update_layout(margin=dict(l=10, r=100, t=50, b=50))
fig.show()
'''

In [ ]:
fig = px.choropleth(
        INFOS_UFS,
        locations="sigla",
        scope='south america',
        center={"lat": -16.95, "lon": -47.78},
        geojson=updated_brazil_states_filtered,
        featureidkey='properties.sigla', # add
        color_continuous_scale="Redor",
        hover_data={"sigla": True}
)
fig.update_geos(fitbounds="geojson", visible=False,framewidth=2, showframe=True)
fig.update_layout(margin=dict(l=10, r=100, t=50, b=50))
fig.show()

In [ ]:
def generate_random_points(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

In [ ]:
br_union_geo = INFOS_UFS['geometry'].unary_union

In [ ]:
gdf_poly = gpd.GeoDataFrame(index=["myPoly"], geometry=[br_union_geo], crs = {'init': 'epsg:4326'},)

In [ ]:
x, y = generate_random_points(br_union_geo, 6_000)

In [ ]:
df = pd.DataFrame()
df['points'] = list(zip(x,y))
df['points'] = df['points'].apply(Point)
gdf_points = gpd.GeoDataFrame(df, geometry='points')
Sjoin = gpd.tools.sjoin(gdf_points, gdf_poly, predicate="within", how='left')

In [ ]:
Sjoin = gpd.tools.sjoin(gdf_points, gdf_poly, predicate="within", how='left')

In [ ]:
# Keep points in "myPoly"
absence = gdf_points[Sjoin.index_right=='myPoly']


In [ ]:
absence['STATUS'] = 0

In [ ]:
OUTPUT_FILE_PATH = "assets/gibf.parquet"

In [ ]:
# Ocurrence species data -> OSD_df
OSD_df = pd.read_parquet(OUTPUT_FILE_PATH)

# Creating Geometry
OSD_df['geometry'] = list(zip(OSD_df["decimalLongitude"], OSD_df["decimalLatitude"]))
OSD_df['geometry'] = OSD_df["geometry"].apply(Point)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    OSD_df,
    crs = {'init': 'epsg:4326'},
    geometry = OSD_df['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)

coord_list = [(x,y) for x,y in zip(OSD_geoframe['geometry'].x , OSD_geoframe['geometry'].y)]

OSD_geoframe['Year'] = OSD_geoframe.eventDate.dt.year
OSD_df = pd.DataFrame(OSD_geoframe)

In [ ]:
OSD_df = OSD_df.rename(columns={'geometry':'points'})
OSD_df['STATUS'] = 1

OSD_df = OSD_df[['points','STATUS']]

In [ ]:
presence_absence = pd.concat([absence, OSD_df])

In [ ]:
presence_absence['decimalLatitude'] = presence_absence['points'].y
presence_absence['decimalLongitude'] = presence_absence['points'].x

In [ ]:
coord_list = [(x,y) for x,y in zip(presence_absence['points'].x , presence_absence['points'].y)]

In [ ]:
fig = px.scatter_mapbox(
    presence_absence,
    lat="decimalLatitude",
    lon="decimalLongitude",
    hover_name="STATUS",
    hover_data=["STATUS"],
    color_discrete_sequence=["fuchsia"],
    color="STATUS",
    zoom=3,
    height=500,
    opacity=1
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

In [ ]:
coord_list = [(x,y) for x,y in zip(presence_absence['points'].x , presence_absence['points'].y)]

In [ ]:
TIF_FILES = "assets/wc2.1_10m_tmax/*.tif"

raster_files = glob.glob(TIF_FILES)
src = rasterio.open(raster_files[0])

for f in raster_files:
    src = rasterio.open(f)
    presence_absence[Path(f).stem] = [x for x in src.sample(coord_list)]
    presence_absence[Path(f).stem] = presence_absence[Path(f).stem].astype('float64')

In [ ]:
presence_absence[['decimalLatitude','decimalLongitude','STATUS']].to_parquet("model/presence_absence.parquet", index=False)